In [1]:
import cv2
import pickle
import numpy as np
from skimage.feature import hog
import matplotlib.pyplot as plt
import math
from collections import deque
import time

MODEL_PATH = 'model_svm.pkl'

with open(MODEL_PATH, 'rb') as f:
    model_data = pickle.load(f)
    svc = model_data['svc']
    scaler = model_data['scaler']
    HOG_PARAMS = model_data['params']

def preprocess_image_fast(img):
    if img is None: 
        return None
    if img.ndim == 3:
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        img_gray = img
    img_gray = cv2.equalizeHist(img_gray)
    return img_gray

def detect_vehicles_ultra_fast(frame, confidence_threshold=1.8):
    if frame is None:
        return []
    
    height, width = frame.shape[:2]
    all_boxes = []
    
    roi_y_start = int(height * 0.3)
    roi_y_end = height - 10
    
    scales = [1.5, 2.0]
    step_size = 32
    
    for scale in scales:
        w = int(64 * scale)
        h = int(64 * scale)
        
        for y in range(roi_y_start, roi_y_end - h, step_size):
            for x in range(0, width - w, step_size):
                if x < width * 0.2 or x > width * 0.8:
                    continue
                
                window = frame[y:y+h, x:x+w]
                if window.shape[0] != h or window.shape[1] != w:
                    continue

                img_processed = preprocess_image_fast(window)
                img_resized = cv2.resize(img_processed, (64, 64))
                
                try:
                    features = hog(img_resized,
                                   orientations=HOG_PARAMS['orientations'],
                                   pixels_per_cell=HOG_PARAMS['pixels_per_cell'],
                                   cells_per_block=HOG_PARAMS['cells_per_block'],
                                   transform_sqrt=HOG_PARAMS['transform_sqrt'],
                                   feature_vector=True,
                                   channel_axis=None)
                    
                    scaled = scaler.transform(features.reshape(1, -1))
                    conf = svc.decision_function(scaled)[0]

                    if conf > confidence_threshold:
                        all_boxes.append([x, y, x+w, y+h, conf])
                except:
                    continue
    
    return all_boxes

def non_max_suppression_fast(boxes, overlap_thresh=0.5):
    if len(boxes) == 0:
        return []
    
    boxes = np.array(boxes)
    if len(boxes) == 0:
        return []
    
    x1 = boxes[:, 0].astype(int)
    y1 = boxes[:, 1].astype(int)
    x2 = boxes[:, 2].astype(int)
    y2 = boxes[:, 3].astype(int)
    conf = boxes[:, 4]
    
    boxes_cv = [[x, y, w, h] for x, y, w, h in zip(x1, y1, x2-x1, y2-y1)]
    
    indices = cv2.dnn.NMSBoxes(
        boxes_cv, 
        conf.tolist(), 
        1.8,
        overlap_thresh
    )
    
    if len(indices) > 0:
        return boxes[indices.flatten()].astype(int)
    else:
        return []

class PathBasedTracker:
    def __init__(self):
        self.next_id = 0
        self.vehicles = {}
        self.counted_ids = set()
        self.path_length = 4
        self.min_distance = 5
    
    def line_intersects_segment(self, line_y, x1, y1, x2, y2):
        if (y1 <= line_y <= y2) or (y2 <= line_y <= y1):
            if y2 == y1:
                return False
            x_at_line = x1 + (x2 - x1) * (line_y - y1) / (y2 - y1)
            return True
        return False
    
    def update(self, detections, line_y):
        current_centers = []
        for box in detections:
            x1, y1, x2, y2, conf = box
            center = ((x1 + x2) // 2, (y1 + y2) // 2)
            current_centers.append((center, box))
        
        updated_vehicles = {}
        counted_now = []
        
        for (center, box) in current_centers:
            cx, cy = center
            matched = False
            best_dist = float('inf')
            best_match_id = None
            
            for vid, data in self.vehicles.items():
                last_center = data['center']
                dist = math.sqrt((cx - last_center[0])**2 + (cy - last_center[1])**2)
                if dist < 80 and dist < best_dist:
                    best_dist = dist
                    best_match_id = vid
            
            if best_match_id is not None:
                vehicle_data = self.vehicles[best_match_id]
                
                vehicle_data['path'].append(center)
                if len(vehicle_data['path']) > self.path_length:
                    vehicle_data['path'].popleft()
                
                if not vehicle_data['counted'] and len(vehicle_data['path']) >= 2:
                    first_pos = vehicle_data['path'][0]
                    last_pos = center
                    
                    if first_pos[1] < line_y and last_pos[1] >= line_y:
                        if self.line_intersects_segment(line_y, first_pos[0], first_pos[1], last_pos[0], last_pos[1]):
                            vehicle_data['counted'] = True
                            self.counted_ids.add(best_match_id)
                            counted_now.append(best_match_id)
                            print(f"   🚗 Vehicle ID {best_match_id} crossed line (top→bottom)! Path: {first_pos[1]} → {last_pos[1]}")
                
                vehicle_data['center'] = center
                vehicle_data['box'] = box
                
                updated_vehicles[best_match_id] = vehicle_data
                matched = True
            else:
                new_vehicle = {
                    'center': center,
                    'path': deque([center], maxlen=self.path_length),
                    'box': box,
                    'counted': False
                }
                updated_vehicles[self.next_id] = new_vehicle
                self.vehicles[self.next_id] = new_vehicle
                self.next_id += 1
        
        filtered_vehicles = {}
        for vid, data in updated_vehicles.items():
            cx, cy = data['center']
            if abs(cy - line_y) < 200 or not data['counted']:
                filtered_vehicles[vid] = data
        
        self.vehicles = filtered_vehicles
        return counted_now

def ultra_fast_real_time_counting():
    print("🚀 Starting ultra-fast real-time vehicle counting with PATH-BASED detection...")
    print("💡 Setup: Make sure OBS Virtual Camera is running")
    print("💡 Press 'q' to quit, 'r' to reset counter")
    
    cap = cv2.VideoCapture(1)
    
    if not cap.isOpened():
        print("❌ Cannot open camera/OBS virtual camera")
        print("💡 Make sure OBS Virtual Camera is started")
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    print(f"📊 Camera Info: {width}x{height} @ {fps}fps")
    print("⚡ Ultra-fast mode enabled!")
    
    tracker = PathBasedTracker()
    total_count = 0
    line_y = height // 2
    
    frame_count = 0
    start_time = time.time()
    fps_counter = 0
    fps_start_time = time.time()
    fps_display = 0
    
    frame_skip = 1
    skip_counter = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Cannot read frame from camera")
            break
        
        frame_count += 1
        skip_counter += 1
        
        if skip_counter > frame_skip:
            skip_counter = 0
            
            raw_boxes = detect_vehicles_ultra_fast(frame, confidence_threshold=1.5)
            
            if len(raw_boxes) > 0:
                final_boxes = non_max_suppression_fast(np.array(raw_boxes), overlap_thresh=0.5)
            else:
                final_boxes = []
            
            counted_ids = tracker.update(final_boxes, line_y)
            total_count += len(counted_ids)
        
        fps_counter += 1
        if time.time() - fps_start_time >= 1.0:
            fps_display = fps_counter
            fps_counter = 0
            fps_start_time = time.time()
        
        result_frame = frame.copy()
        
        cv2.line(result_frame, (0, line_y), (width, line_y), (0, 255, 255), 3)
        cv2.putText(result_frame, "COUNTING LINE", (width-200, line_y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
        for vid, data in tracker.vehicles.items():
            if 'box' in data and abs(data['center'][1] - line_y) < 150:
                x1, y1, x2, y2, conf = data['box']
                
                if data['counted']:
                    color = (0, 0, 255)
                    thickness = 2
                elif conf > 3.0:
                    color = (0, 255, 0)
                    thickness = 2
                else:
                    color = (0, 165, 255)
                    thickness = 1
                
                cv2.rectangle(result_frame, (x1, y1), (x2, y2), color, thickness)
                
                center = data['center']
                cv2.circle(result_frame, center, 3, (255, 255, 0), -1)
                
                if len(data['path']) > 1:
                    path_points = list(data['path'])
                    for i in range(1, len(path_points)):
                        cv2.line(result_frame, path_points[i-1], path_points[i], (255, 255, 0), 1)
                
                if data['counted']:
                    cv2.putText(result_frame, "COUNTED", (x1, y2+15),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        
        cv2.rectangle(result_frame, (width-280, 0), (width, 145), (0, 0, 0), -1)

        cv2.putText(result_frame, f"COUNT: {total_count}", 
                   (width-270, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
        cv2.putText(result_frame, f"Active: {len(tracker.vehicles)}", 
                   (width-270, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
        cv2.putText(result_frame, f"FPS: {fps_display}", 
                   (width-270, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)  # ← BARIS BARU FPS
        cv2.putText(result_frame, f"Press 'q' to quit", 
                   (width-270, 115), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        cv2.putText(result_frame, "'r': reset | 's': save", 
                   (width-270, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        
        cv2.imshow('Path-Based Vehicle Counting', result_frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            print("🛑 Quitting...")
            break
        elif key == ord('r'):
            total_count = 0
            tracker = PathBasedTracker()
            print("🔄 Counter reset!")
        elif key == ord('s'):
            elapsed_time = time.time() - start_time
            print(f"💾 Results saved!")
            print(f"   Total vehicles: {total_count}")
            print(f"   Processing time: {elapsed_time:.1f}s")
            print(f"   Average: {total_count/(elapsed_time/60):.1f} vehicles/minute")
    
    cap.release()
    cv2.destroyAllWindows()
    
    elapsed_time = time.time() - start_time
    print(f"\n✅ Path-based counting completed!")
    print(f"   Total frames processed: {frame_count}")
    print(f"   Total vehicles counted: {total_count}")
    print(f"   Processing time: {elapsed_time:.1f} seconds")
    print(f"   Average: {total_count/(elapsed_time/60):.1f} vehicles/minute")
    
    return total_count

if __name__ == "__main__":
    print("🎯 Path-Based Real-time Vehicle Counting - SOLVED CROSSING ISSUE")
    print("=" * 70)
    
    print("⚡ Optimizations applied:")
    print("   • Path-based crossing detection")
    print("   • Only 2 scales: [1.5, 2.0]")
    print("   • Large step size: 32")
    print("   • Aggressive ROI: 30% from top")
    print("   • Fast NMS using OpenCV")
    print("   • Limited active vehicles")
    print()
    print("🎯 Features:")
    print("   • Detects vehicles crossing from ANY speed")
    print("   • Shows vehicle paths for debugging")
    print("   • Accurate counting for both slow and fast vehicles")
    print()
    
    total = ultra_fast_real_time_counting()
    print(f"\n🎯 Final Count: {total} vehicles")

c:\Users\hosea\anaconda3\envs\computer_vision\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\hosea\anaconda3\envs\computer_vision\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


🎯 Path-Based Real-time Vehicle Counting - SOLVED CROSSING ISSUE
⚡ Optimizations applied:
   • Path-based crossing detection
   • Only 2 scales: [1.5, 2.0]
   • Large step size: 32
   • Aggressive ROI: 30% from top
   • Fast NMS using OpenCV
   • Limited active vehicles

🎯 Features:
   • Detects vehicles crossing from ANY speed
   • Shows vehicle paths for debugging
   • Accurate counting for both slow and fast vehicles

🚀 Starting ultra-fast real-time vehicle counting with PATH-BASED detection...
💡 Setup: Make sure OBS Virtual Camera is running
💡 Press 'q' to quit, 'r' to reset counter
📊 Camera Info: 640x480 @ 30fps
⚡ Ultra-fast mode enabled!
🛑 Quitting...

✅ Path-based counting completed!
   Total frames processed: 30
   Total vehicles counted: 0
   Processing time: 7.8 seconds
   Average: 0.0 vehicles/minute

🎯 Final Count: 0 vehicles
